Enterprise GenAI Adoption & Workforce Impact Dataset
- "This dataset originates from a multi-year enterprise survey conducted across industries and countries. It focuses on the organizational effects of adopting Generative AI tools such as ChatGPT, Claude, Gemini, Mixtral, LLaMA, and Groq. The dataset captures detailed metrics on job role creation, workforce transformation, productivity changes, and employee sentiment." ~ Kaggle.com
- Over the last couple years, companies across almost all industries have pivoted to implementing Generative AI models and tools in their day-to-day operations to assist in employee productivity and streamline repetitive tasks. The automation that comes with these tools increases efficiency within the company, decreasing time elapsed per task. These advantages motivated the world to shift into this adjustment and this dataset provides a comprehensive overview of the integration of GenAI in the workforce.

Analysis of Features:
- Company Name
  - Simply provides the name of the company per row.
- Industry
  - Notes the specific industry that the company is in. This feature provides valuable insight on the sector of the workforce that is integrating GenAI in their operations. This can be interesting because you'll see the wide range of different industries that are adopting GenAI, sometimes industries that you don't expect to see.
- Country
  - Provides the country of the company per row.
- GenAI Tool
  - Provides the specific GenAI tool each company decided to integrate into their operations. Each company utilizes an individual tool so they can obtain licensing for the tool and provide it for each employee to use. Companies often seek out a different tool based on their needs and industry they're in, as tools are trained to be more efficient in certain aspects.
- Adoption Year
  - The year that the company adopting the GenAI tool.
- Number of Employees Impacted
  - Staff count pertaining to the amount of employees impacted by the adoption of the GenAI tool within the company. Necessary detail as some employees won't be affected by the GenAI tool, such as the maintenance team, for example. Employees affected are the employees that either had their jobs automated or that use the GenAi tool to streamline tasks and assist them in their work.
- New Roles Created
  - Number of AI-driven roles that were created within the company. With such a major aspect change within the workforce, there has to be some sort of support system and development team for the tool to ensure seamless integration and ease-of-use for employees across all divisions. Not everyone has experience using GenAI tools so this is a crucial adjustment for companies.
- Training Hours Provided
  - Investment of training hours the company provided to staff. Similar to the last feature, not everyone has experience using GenAI tools so training is a necessity for companies looking to implement this feature into their operations on a daily basis. While it is a tool, the misuse of it can negatively impact productivity within the company.
- Productivity Change (%)
  - Percent change in productivity since the adoption of the GenAI tool within the company. This features has a wide range, displaying the effectiveness of GenAI tools in different industries and companies.
- Employee Sentiment
  - Provides the textual feedback from the employees of the companies. This is qualitative data essential in understanding the responsiveness of workers with the new implementation of tools in their company. Understanding how the employees feel says a lot about the trajectory of AI tools in the workforce.
 
Most Important Features for Model Training:
- Industry
- GenAI tool
- Number of Employees Impacted
- New Roles Created
- Productivity Change
- Employee Sentiment

Our reasoning for choosing these specific features for training our AI model is due to their insight on the trajectory of AI tools in the workforce and how we can use them to analyze the pros and cons of Generative AI in the workforce across industries. As the rise of AI continues to trend upwards, its important to analyze its impact and the feedback from employees integrating it into their day-to-day operations.

In [4]:
#Visualization of Data
import numpy as np
import pandas as pd
import matplotlib as plt

df_genai = pd.read_csv('genaidataset.csv')

,Company Name,Industry,Country,GenAI Tool,Adoption Year,Number of Employees Impacted,New Roles Created,Training Hours Provided,Productivity Change (%),Employee Sentiment
0,Davis LLC Pvt. Ltd.,Healthcare,USA,Mixtral,2022,5277,8,657,25.2,"Productivity increased, but there's anxiety ab..."
1,"Roberts, Holland and Bradshaw Pvt. Ltd.",Telecom,South Africa,Claude,2023,18762,17,23021,27.5,"We now finish tasks faster, but some older emp..."
2,Roman Inc Pvt. Ltd.,Advertising,India,Gemini,2023,11307,17,4680,11.5,"Productivity increased, but there's anxiety ab..."
3,Nguyen-Strickland Pvt. Ltd.,Technology,UK,Groq,2023,18834,12,1750,7.0,"AI helped me reduce repetitive tasks, but lear..."
4,Jackson PLC Pvt. Ltd.,Hospitality,UK,LLaMA,2024,8364,22,3925,2.5,"Job roles have shifted a lot, which is both go..."
5,Forbes Ltd Pvt. Ltd.,Entertainment,Switzerland,Gemini,2022,16082,16,11485,19.9,New roles are exciting; transition was smoothe...
6,"Barker, Williams and Lynch Pvt. Ltd.",Telecom,South Africa,Mixtral,2022,994,11,2593,6.6,"Job roles have shifted a lot, which is both go..."
7,"Wheeler, Warner and Brown Pvt. Ltd.",Advertising,USA,Groq,2022,16429,16,11692,20.4,Collaboration improved; documentation and meet...
8,"Ramirez, Wells and Cochran Pvt. Ltd.",Technology,South Korea,Gemini,2022,12952,7,22484,29.5,There's concern that AI will replace some of u...
9,Morris Inc Pvt. Ltd.,Legal Services,Switzerland,ChatGPT,2022,4144,24,20689,33.3,I love using AI—it makes my job easier and mor...
